In [1]:
# Device configuration
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
import os
import utils
from mobilenet import MobileNet

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define directories
base_dir = '..\\fruits_vegetables_ds'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_df = utils.create_df(train_dir)
validation_df = utils.create_df(validation_dir)
test_df = utils.create_df(test_dir)

classes = [class_name for class_name in os.listdir(train_dir)]

# Create datasets and dataloaders
train_dataset = utils.CustomDataset(train_df, transform=utils.preprocess)
validation_dataset = utils.CustomDataset(validation_df, transform=utils.preprocess)
test_dataset = utils.CustomDataset(test_df, transform=utils.preprocess)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


mobilenet = MobileNet(num_classes=len(classes))

mobilenet = mobilenet.to(device)

# Load the state dictionary from the pretrained model
#state_dict = torchMobilenet.state_dict()

# Exclude the classifier's parameters from the state dictionary
#state_dict = {k: v for k, v in state_dict.items() if not k.startswith('classifier')}

# Load the modified state dictionary into the another model
#mobilenet.load_state_dict(state_dict, strict=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

# Testing function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(test_loader.dataset), correct / total

best_val_loss = float('inf')
best_model_state = None
num_epochs = 20
patience = 10
count = 0


for epoch in range(num_epochs):
    train_loss = train(mobilenet, train_loader, criterion, optimizer, device)
    val_loss, val_acc = test(mobilenet, validation_loader, criterion, device)
    print(f'Epoch [{epoch}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = mobilenet.state_dict()

# Save the best model
if best_model_state is not None:
    torch.save(mobilenet.state_dict(), 'best_model.pth')
    print('Best model saved.')

# Load the best model state
if best_model_state is not None:
    mobilenet.load_state_dict(best_model_state)
    print('Best model loaded.')

# Test the best model
mobilenet.eval()
test_loss, test_acc = test(mobilenet, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')


Epoch [0], Train Loss: 3.2615, Val Loss: 2.9376, Val Acc: 0.1880
Epoch [1], Train Loss: 2.6979, Val Loss: 2.2216, Val Acc: 0.2821
Epoch [2], Train Loss: 2.4097, Val Loss: 2.1566, Val Acc: 0.2991
Epoch [3], Train Loss: 2.2504, Val Loss: 1.8224, Val Acc: 0.4359
Epoch [4], Train Loss: 2.1129, Val Loss: 1.6674, Val Acc: 0.4245
Epoch [5], Train Loss: 2.0356, Val Loss: 1.7252, Val Acc: 0.4217
Epoch [6], Train Loss: 1.9043, Val Loss: 1.4063, Val Acc: 0.5499
Epoch [7], Train Loss: 1.8004, Val Loss: 1.4348, Val Acc: 0.5214
Epoch [8], Train Loss: 1.6964, Val Loss: 1.1192, Val Acc: 0.6239
Epoch [9], Train Loss: 1.6167, Val Loss: 1.0715, Val Acc: 0.6553
Epoch [10], Train Loss: 1.4534, Val Loss: 0.9178, Val Acc: 0.6809
Epoch [11], Train Loss: 1.3684, Val Loss: 0.8133, Val Acc: 0.7350
Epoch [12], Train Loss: 1.2974, Val Loss: 0.8801, Val Acc: 0.6695
Epoch [13], Train Loss: 1.1988, Val Loss: 0.7274, Val Acc: 0.7892
Epoch [14], Train Loss: 1.0604, Val Loss: 0.7019, Val Acc: 0.7835
Epoch [15], Train Lo

In [16]:
#exporting the model

torch_model = mobilenet.cpu()
torch_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(torch_model,              # model being run
                  torch_input,                         # model input (or a tuple for multiple inputs)
                  "vegetable_classifier.onnx",     # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [17]:
import onnxruntime
import numpy as np

# Load the exported ONNX model
ort_session = onnxruntime.InferenceSession("vegetable_classifier.onnx")

# Initialize lists to store predictions from PyTorch and ONNX Runtime
pytorch_predictions = []
onnx_predictions = []

# Test the model with each batch in the test dataset
for batch in test_loader:
    x, y = batch

    # Compute PyTorch output prediction
    pytorch_output = mobilenet(x).detach().cpu().numpy()

    # Compute ONNX Runtime output prediction
    ort_inputs = {ort_session.get_inputs()[0].name: x.numpy()}  # Convert to numpy array
    ort_outs = ort_session.run(None, ort_inputs)
    onnx_output = ort_outs[0]

    # Append predictions to lists
    pytorch_predictions.append(pytorch_output)
    onnx_predictions.append(onnx_output)

# Concatenate lists to numpy arrays
pytorch_predictions = np.concatenate(pytorch_predictions)
onnx_predictions = np.concatenate(onnx_predictions)

# Compare PyTorch and ONNX Runtime results
np.testing.assert_allclose(pytorch_predictions, onnx_predictions, rtol=1e-03, atol=1e-05)

print("Model has been tested against the test dataset using both PyTorch and ONNX Runtime, and the results are consistent.")

Model has been tested against the test dataset using both PyTorch and ONNX Runtime, and the results are consistent.


In [22]:
from PIL import Image

# Load the exported ONNX model
ort_session = onnxruntime.InferenceSession("other_classifier.onnx")

# Load and preprocess a single image
image_test_path_dir = '..\\misc_ds\\validation\\rice'

classes = [class_name for class_name in os.listdir('..\\misc_ds\\validation')]

for filename in os.listdir(image_test_path_dir):
    # Load and preprocess the image
    img_path = os.path.join(image_test_path_dir, filename)
    img = Image.open(img_path)
    transformed_img = utils.preprocess(img)  # Assuming utils.preprocess performs the necessary transformations

    # Add a batch dimension to the image tensor
    transformed_img = transformed_img.unsqueeze(0)  # Add batch dimension

    # Compute ONNX Runtime output prediction
    ort_inputs = {ort_session.get_inputs()[0].name: transformed_img.numpy()}  # Convert to numpy array
    ort_outs = ort_session.run(None, ort_inputs)
    onnx_output = ort_outs[0]

    # Get the predicted class index
    max_index = np.argmax(onnx_output)
    print("ONNX Runtime output prediction:", classes[max_index])

ONNX Runtime output prediction: rice
ONNX Runtime output prediction: flour
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: flour
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: flour
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
ONNX Runtime output prediction: rice
